# Ablation: Personality Hassles notebook


## Section 1: Reading the dataset and data preparation

In [355]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from scaler import CustomScaler

In [356]:
filenames = ['General_hassles', 'Inner_concerns', 'Financial_concerns', 'Time_Pressures']
extension = '.csv'
dfs = {}
test_dfs = {}
for filename in filenames:
    df = pd.read_csv(f'../data/category/{filename}.csv')
    df = df.sample(frac = 1, random_state = 42)

    # df['Hassle'] = LabelEncoder().fit_transform(df['Hassle'])

    # scaler_O = MinMaxScaler(feature_range=(10, 50))
    # scaler_C = MinMaxScaler(feature_range=(9, 45))
    # scaler_E = MinMaxScaler(feature_range=(8, 40))
    # scaler_A = MinMaxScaler(feature_range=(9, 45))
    # scaler_N = MinMaxScaler(feature_range=(8, 40))

    # # Fit and transform specific columns
    # df['O'] = scaler_O.fit_transform(df[['O']])
    # df['C'] = scaler_C.fit_transform(df[['C']])
    # df['E'] = scaler_E.fit_transform(df[['E']])
    # df['A'] = scaler_A.fit_transform(df[['A']])
    # df['N'] = scaler_N.fit_transform(df[['N']])
    
    #drop age and gender for now
    df.drop(columns = ['Age', 'Gender'], inplace = True)

    dfs[filename] = df

## Section 2: Column Transformer

In [357]:
from sklearn.compose import ColumnTransformer

### 2.1 Scaler

In [358]:
ct1 = ColumnTransformer(
    [
        ('oScaler', CustomScaler(data_range = [10, 50]), ['O']),
        ('cScaler', CustomScaler(data_range = [9, 45]), ['C']),
        ('eScaler', CustomScaler(data_range = [8, 40]), ['E']),
        ('aScaler', CustomScaler(data_range = [9, 45]), ['A']),
        ('nScaler', CustomScaler(data_range = [8, 40]), ['N']),
    ],
)

In [359]:
# This function dynamically creates a columnt ransformer object and sets in on the piepline

# Takes in the array of columns and pipe
def create_column_transformer(cols, pipe):
    scaler_list = []
    for col in cols:
        if(col == 'O'):
            scaler = ('oScaler', CustomScaler(data_range = [10, 50]), ['O'])
        elif(col == 'C'):
            scaler = ('cScaler', CustomScaler(data_range = [9, 45]), ['C'])
        elif(col == 'E'):
            scaler = ('eScaler', CustomScaler(data_range = [8, 40]), ['E'])
        elif(col == 'A'):
            scaler = ('aScaler', CustomScaler(data_range = [9, 45]), ['A'])
        elif(col == 'N'):
            scaler = ('nScaler', CustomScaler(data_range = [8, 40]), ['N'])
        else:
            return ValueError;
        scaler_list.append(scaler)
    
    ct = ColumnTransformer(transformers = scaler_list)

    pipe.steps[0] = ('transformer', ct)

### 2.2 Encoder

In [360]:
ct2 = ColumnTransformer(
    [
        ('encoder', LabelEncoder(), ['Hassle'])
    ]
)

## Section 3: Model and Hyperparameters Initialization

### 3.1 Model Initialization

In [361]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR, NuSVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor


In [362]:
linreg = LinearRegression()
ridge = Ridge()
lasso = Lasso()
elasticnet = ElasticNet()

svr = SVR(verbose = 0)
nusvr = NuSVR(verbose = 0)
lsvr = LinearSVR(verbose = 0)

dt = DecisionTreeRegressor()

rfr = RandomForestRegressor( random_state = 42)
gbr = GradientBoostingRegressor( random_state = 42)
vr = VotingRegressor(estimators = RandomForestRegressor(random_state = 42))
br = BaggingRegressor( random_state = 42)
abr = AdaBoostRegressor(random_state = 42)

nn = MLPRegressor(random_state = 42, verbose = 0)


### 3.2 Pipleline initialization

In [363]:
from sklearn.pipeline import Pipeline

In [364]:
pipe = Pipeline([('preprocessor1', ct1), ('regressor', ridge)])

### 3.3 Hyperparams Initialization

In [365]:
hyperparameters = [
    {
        'regressor': [linreg]
    },
    {
        'regressor__alpha': [0, 0.5, 1, 2],
        'regressor__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'regressor__tol': [0.0001, 0.00001, 0.001],
        'regressor': [ridge]
    },
    {
        'regressor__alpha': [0, 0.5, 1, 2],
        'regressor__tol': [0.0001, 0.00001, 0.001],
        'regressor__selection': ['cyclic', 'random'],
        'regressor': [lasso]
    },
    {
        'regressor__alpha': [0, 0.5, 1, 2],
        'regressor__l1_ratio': [0.5, 0.05, 0.005],
        'regressor__tol': [0.0001, 0.00001, 0.001],
        'regressor__selection': ['cyclic', 'random'],
        'regressor': [elasticnet]
    },
    # {
    #     'regressor__kernel': ['rbf', 'linear', 'poly'],
    #     'regressor__degree': [3, 7, 11],
    #     'regressor__C': [1.0, 1.5, 0.5],
    #     'regressor__epsilon': [0.1, 0.01, 0.001],
    #     'regressor': [svr]
    # },
    # {
    #     'regressor__kernel': ['rbf', 'linear', 'poly'],
    #     'regressor__degree': [3, 7, 11],
    #     'regressor__C': [1.0, 1.5, 0.5],
    #     'regressor__nu': [0.01, 0.5, 1],
    #     'regressor': [nusvr]
    # },
    # {
    #     'regressor__epsilon': [0.0, 0.5, 0.05],
    #     'regressor__C': [1.0, 0.5, 1.5],
    #     'regressor__tol': [0.0001, 0.00001, 0.001],
    #     'regressor__loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'],
    #     'regressor': [lsvr]
    # },
    {
        'regressor__criterion': ['squared_error', 'poisson', 'friedman_mse', 'absolute_error'],
        'regressor__splitter': ['best', 'random'],
        'regressor__max_depth': [4, 8, 12],
        'regressor__min_samples_split': [4, 8, 12],
        'regressor': [dt]
    },
    {
        'regressor__n_estimators': [10, 15, 20, 25, 45, 55, 75, 100, 125],
        'regressor__criterion': ['mse', 'mae', 'friedman_mse', 'poisson'],
        'regressor__max_depth': [4, 5, 8, 12],
        'regressor__min_samples_split': [4, 8, 12],
        'regressor__bootstrap': [True, False],
        'regressor': [rfr]
    },
    {
        'regressor__n_estimators': [10, 15, 20, 25, 45, 55, 75, 100, 125],
        'regressor__max_depth': [4, 8, 12],
        'regressor__min_samples_split': [4, 8, 12],
        'regressor__learning_rate': [1.0, 0.1, 0.01],
        'regressor__loss': ['ls', 'lad', 'huber', 'quantile'],
        'regressor': [gbr]
    },
    {
        'regressor__base_estimator': [
            DecisionTreeRegressor(random_state = 42, max_depth = 5), 
            DecisionTreeRegressor(random_state = 42, max_depth = 8), 
            DecisionTreeRegressor(random_state = 42, max_depth = 12), 
            RandomForestRegressor(random_state = 42,  n_estimators = 10),
            RandomForestRegressor(random_state = 42,  n_estimators = 15),
            RandomForestRegressor(random_state = 42,  n_estimators = 20),
            ],
        'regressor__n_estimators': [10, 20, 5],
        'regressor__bootstrap': [True, False],
        'regressor': [br]
    },
    {
        'regressor__base_estimator': [
            DecisionTreeRegressor(random_state = 42, max_depth = 5), 
            DecisionTreeRegressor(random_state = 42, max_depth = 8), 
            DecisionTreeRegressor(random_state = 42, max_depth = 12), 
            RandomForestRegressor(random_state = 42, n_estimators = 10), 
            RandomForestRegressor(random_state = 42, n_estimators = 15), 
            RandomForestRegressor(random_state = 42, n_estimators = 20), 
            ],
        'regressor__n_estimators': [25, 50, 75],
        'regressor__learning_rate': [1.0, 0.1, 0.05],
        'regressor__loss': ['linear', 'square', 'exponential'],
        'regressor': [abr]
    },
    {
        'regressor__hidden_layer_sizes' : [(20, 20, 20, 20, 20), (15, 15, 15, 15, 15, 15, 15), (10, 10, 10, 10, 10, 10, 10, 10, 10, 10)],
        'regressor__alpha': [0.0001, 0.001, 0.00001],
        'regressor__learning_rate_init' : [0.0001, 0.001, 0.01],
        'regressor__learning_rate': ['constant', 'invscaling', 'adaptive'],
        'regressor__solver' : ['adam', 'sgd', 'lbfgs'],
        'regressor__activation' : ['relu', 'tanh'],
        'regressor': [nn]
    },

]


## Section 4: Model Training and Prediction

PIPELINE: Iterate through each df -> Iterate through each possible combination of OCEAN -> initialize a grid search -> train models -> get predictions -> store subset, r2 scores and best model in an array -> after each iteration of df -> get the highest r2 value in the array

In [366]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from pathlib import Path
import joblib
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
import itertools

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [367]:
for df_name, df in dfs.items():
    # Trying ablation study for General hassles
    feature_columns = df.columns[:-2]

    # Initialize a subset, r2 scores and models array
    subsets = []
    r2scores = []
    models = []
    params = []


    # Iterate through each possible subset
    for L in range(1, len(feature_columns) + 1):
        for subset in itertools.combinations(feature_columns, L):
            selected_features = list(subset)

            # Initialize X and y
            X = df[selected_features]
            X['Hassle'] = df['Hassle']
            y = df.iloc[:, -1]

            # Train test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


            # Change the columns in column transformer
            create_column_transformer(subset, pipe)

            # Iterate through each model and the hyperparams
            grid = GridSearchCV(pipe, hyperparameters)
            grid.fit(X_train, y_train)
            # grid.best_params_

            # Get best model and hyperparams
            best_model = grid.best_estimator_
            best_params = grid.best_params_

            # Make predictions
            y_pred = best_model.predict(X_test)

            # Get r2 score
            r2 = r2_score(y_test, y_pred)

            #Save subsets, r2score, params and model
            subsets.append(subset)
            r2scores.append(r2)
            params.append(best_params)
            models.append(best_model)

    # Get the index of the highest r2 score on the current dataset
    highest_r2_index = r2scores.index(max(r2scores))

    # Get the corresponding subset, params and model
    corr_subset = subsets[highest_r2_index]
    corr_params = params[highest_r2_index]
    corr_model = models[highest_r2_index]

    # Show the results
    print(df_name)
    print("Subset: ", corr_subset)
    print("R2 score: ", max(r2scores))
    display(corr_model)

    # Define directory where params and model will be saved
    params_directory = Path('models/specs/')
    model_directory = Path('models/model/')

    # Get model name
    name = type(corr_model).__name__

    # Generate file
    params_file = params_directory / f"{df_name}_{name}_hyperparams.pkl"
    model_file = model_directory / f"{df_name}_{name}_model.pkl"

    # Save hyperparams
    with open(params_file, 'wb') as f:
        joblib.dump(best_params, f)

    # Save Model
    with open(model_file, 'wb') as f:
        joblib.dump(best_model, f)



KeyboardInterrupt: 

In [ ]:
# # Trying ablation study for General hassles
# feature_columns = dfs['General_hassles'].columns[:-2]

# # Initialize a subset, r2 scores and models array
# subsets = []
# r2scores = []
# models = []
# params = []

# for L in range(1, len(feature_columns) + 1):
#     for subset in itertools.combinations(feature_columns, L):
#         selected_features = list(subset)

#         # Initialize X and y
#         X = df[selected_features]
#         X['Hassle'] = df['Hassle']
#         y = df.iloc[:, -1]

#         # Train test split
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#         # Change the columns in column transformer
#         create_column_transformer(subset, pipe)

#         # Iterate through each model and the hyperparams
#         grid = GridSearchCV(pipe, hyperparameters)
#         grid.fit(X_train, y_train)
#         # grid.best_params_

#         # Get best model and hyperparams
#         best_model = grid.best_estimator_
#         best_params = grid.best_params_

#         # Get best model name
#         name = type(best_model).__name__

#         # Make predictions
#         y_pred = best_model.predict(X_test)

#         # Get r2 score
#         r2 = r2_score(y_test, y_pred)

#         #Save subsets, r2score, params and model
#         subsets.append(subset)
#         r2scores.append(r2)
#         params.append(best_params)
#         models.append(best_model)

# # Get the index of the highest r2 score
# highest_r2_index = r2scores.index(max(r2scores))

# # Get the corresponding subset, params and model
# corr_subset = subsets[highest_r2_index]
# corr_params = params[highest_r2_index]
# corr_model = models[highest_r2_index]

# #Show the results
# print("Subset: ", corr_subset)
# print("R2 score: ", max(r2scores))
# print(corr_model)

# # Define directory where model will be saved


# # Generate filename

# # Save Model

# # Save hyperparams
